In [3]:
import os
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

In [4]:
# Parameters
IMG_SIZE: tuple[int, int] = (256, 256)
BATCH_SIZE: int = 16
EPOCHS: int = 30
BASE_DIR: str = "./brain_scans" # Root folder containing glioma, meningioma, and pituitary_tumor folders

In [6]:
# Data Augmentation and Image Loading
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.2 # 20% data for validation
)

In [7]:
train_gen = datagen.flow_from_directory(
    BASE_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training" # Set for training data
)

Found 2452 images belonging to 3 classes.


In [8]:
val_gen = datagen.flow_from_directory(
    BASE_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation" # Set for validation data
)

Found 612 images belonging to 3 classes.


In [9]:
# Load Pre-trained ResNet50 Model (without the top layers)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 29s 0us/step


In [10]:
# Freezing the convolutional base
for layer in base_model.layers:
    layer.trainable = False

In [11]:
# Adding new custom layers
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.5)(x)
output = Dense(3, activation="softmax")(x) # 3 classes for glioma, meningioma, pituitary tumor

2024-09-11 15:08:57.902651: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 268435456 exceeds 10% of free system memory.
2024-09-11 15:08:58.234805: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 268435456 exceeds 10% of free system memory.
2024-09-11 15:08:58.346535: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 268435456 exceeds 10% of free system memory.


In [12]:
model = Model(inputs=base_model.input, outputs=output)

In [13]:
# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

In [14]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

In [15]:
# Train the model
history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

Epoch 1/30


2024-09-11 15:11:39.999876: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 268435456 exceeds 10% of free system memory.
/home/penzulo/.cache/pypoetry/virtualenvs/neurosight-li8zIHSD-py3.11/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-09-11 15:11:52.968917: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 268435456 exceeds 10% of free system memory.


154/154 ━━━━━━━━━━━━━━━━━━━━ 1011s 6s/step - accuracy: 0.3935 - loss: 10.4024 - val_accuracy: 0.4657 - val_loss: 1.0854 - learning_rate: 0.0010
Epoch 2/30
154/154 ━━━━━━━━━━━━━━━━━━━━ 919s 6s/step - accuracy: 0.4713 - loss: 1.0807 - val_accuracy: 0.4657 - val_loss: 1.0711 - learning_rate: 0.0010
Epoch 3/30
154/154 ━━━━━━━━━━━━━━━━━━━━ 897s 6s/step - accuracy: 0.4563 - loss: 1.0728 - val_accuracy: 0.4657 - val_loss: 1.0637 - learning_rate: 0.0010
Epoch 4/30
154/154 ━━━━━━━━━━━━━━━━━━━━ 849s 5s/step - accuracy: 0.4615 - loss: 1.0623 - val_accuracy: 0.4657 - val_loss: 1.0597 - learning_rate: 0.0010
Epoch 5/30
154/154 ━━━━━━━━━━━━━━━━━━━━ 889s 6s/step - accuracy: 0.4787 - loss: 1.0541 - val_accuracy: 0.4657 - val_loss: 1.0578 - learning_rate: 0.0010
Epoch 6/30
154/154 ━━━━━━━━━━━━━━━━━━━━ 831s 5s/step - accuracy: 0.4523 - loss: 1.0624 - val_accuracy: 0.4657 - val_loss: 1.0569 - learning_rate: 0.0010
Epoch 7/30
154/154 ━━━━━━━━━━━━━━━━━━━━ 883s 6s/step - accuracy: 0.4674 - loss: 1.0569 - va

In [17]:
model.save("brain_tumor_classifier.keras")

In [20]:
val_loss, val_accuracy = model.evaluate(val_gen)

39/39 ━━━━━━━━━━━━━━━━━━━━ 141s 4s/step - accuracy: 0.4844 - loss: 1.0472


In [21]:
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

Validation Accuracy: 46.57%
